# Section: BLAS/LAPACK - Linear Equations


Adapted from: [https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/blas_lapack/linear_equations](https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/blas_lapack/linear_equations)

## This program demonstrates solving linear algebra equations in Fortran.

### Linear Equation Problem

In this notebook we will use Fortran to solve a system of linear equations of the form:

$$
\Large A \mathbf{x} = \mathbf{b}
$$
where: <br>


$$
\Large A =
\left[
\begin{array}{cccc}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \vdots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{array}
\right]
$$

$$
\Large \mathbf{x} =
\left[
\begin{array}{c}
x_{1}  \\
x_{2}  \\
\vdots \\
x_{n}
\end{array}
\right]
$$

$$
\Large \mathbf{b} = 
\left[
\begin{array}{c}
b_{1}  \\
b_{2}  \\
\vdots \\
b_{n}
\end{array}
\right]
$$

The approach will be to have a main program that reads in as parameters the number of equations, the matrix $\mathbf{A}$ and the vector $\mathbf{x}$ as text files.  The main program is called *solve_equations.f90*.  It makes use of the LAPACK linear equation solver routine called **DGESV**.  Information on this routine can be found at [LAPACK - DGESV](https://netlib.org/lapack/explore-html/d7/d3b/group__double_g_esolve_ga5ee879032a8365897c3ba91e3dc8d512.html#ga5ee879032a8365897c3ba91e3dc8d512).

The main program makes use of a module called *linalg_mod.f90*, which contains subroutines to read and write matrices and arrays to and from text files.

Finally, there is a helper program called *generate_array.f90* which has a subroutine that gets arguments from the command line and generates either of vector or matrix of specified size.  Every element in the array or matrix is a random number.

## Explanation of the Programs

The following explanation is given by "Code Copilot" within ChatGPT 4.

### Explaining `generate_array.f90`

The Fortran program `generate_array.f90` serves to generate either a vector or a matrix with random elements, depending on the command line arguments provided. It uses routines from a module named `linalg_mod` to handle array creation and output. Below is an explanation of the program structure and its key components:

#### Program Structure

1. **Module Usage**
   - `iso_fortran_env`: Utilized for standard environment variables and constants, specifically `error_unit` for error messages and `DP` alias for `REAL64`, indicating double precision.
   - `linalg_mod`: Externally defined module used here for `write_array` and `generate_random_array`, which handle array operations.

2. **Variable Declarations**
   - `dimensions`: An allocatable integer array to store dimensions provided via command-line arguments.
   - `vector`: An allocatable vector of type `real(kind=DP)`.
   - `matrix`: An allocatable 2D array of type `real(kind=DP)`.

3. **Main Execution Logic**
   - It begins by fetching command-line arguments into `dimensions`.
   - Based on the size of `dimensions`, it either generates a 1D vector or a 2D matrix:
     - For a single dimension, a vector of random elements is created and written.
     - For two dimensions, a matrix is created and written.
   - If neither one nor two dimensions are provided, it outputs an error indicating that only rank 1 and 2 arrays are supported.

4. **Error Handling**
   - The program checks for proper memory allocation and command-line argument processing, with appropriate error messages and stops if any issues occur.

#### Subroutine: `get_arguments`

- **Purpose**: Fetches command-line arguments to determine the size of the array to be generated.
- **Process**:
  - Determines the number of arguments (`rank`).
  - Allocates the `dimensions` array based on `rank`.
  - Reads each command argument into `dimensions`, converting from string to integer.
  - Handles errors in conversion and allocation, stopping the program if necessary.

### Key Functionalities

- **Array Generation**: Depending on the input dimensions, it generates either a vector or a matrix with random values.
- **Modular Design**: Leverages external modules for specific functionalities, which simplifies the main program.
- **Flexibility**: The ability to handle both vectors and matrices makes the program versatile for different kinds of numerical data generation tasks.

### Usage

- The program is executed with command-line arguments specifying the dimensions of the array to generate. For example:
  - `./generate_array 10` would generate a vector of size 10.
  - `./generate_array 3 4` would generate a 3x4 matrix.

Now, let's move on to read and explain the `solve_equations.f90` file.

### Explaining `solve_equations.f90`

The Fortran program `solve_equations.f90` is designed to solve a system of linear equations using LAPACK's `dgesv` subroutine, which solves systems of linear equations \( AX = B \) by performing LU decomposition with partial pivoting. Here's a breakdown of its key components and functionalities:

#### Program Structure

1. **Module Usage**
   - `iso_fortran_env`: Standard intrinsic module providing access to the `error_unit` and `DP` (double precision kind type alias).
   - `linalg_mod`: Custom module providing `read_array` and `write_array` functions, used to handle input/output operations for matrices and vectors.

2. **Variable Declarations**
   - `nr_eqns`: Number of equations in the system, derived from command-line arguments.
   - `status`, `info`: Variables for checking statuses of operations; `info` is used specifically with `dgesv`.
   - `matrix_file_name`, `vector_file_name`: File names for storing matrix \( A \) and vector \( B \).
   - `b`: Vector \( B \) in the equation \( AX = B \).
   - `pivot`: Integer array for pivot indices from the LU decomposition.
   - `A`: Matrix \( A \) of the system.

3. **Explicit Interface for `dgesv`**
   - The subroutine's interface is explicitly defined to ensure correct passing of arguments:
     - `n`, `nrhs`, `a`, `lda`, `ipiv`, `b`, `ldb`, and `info` describe the system dimensions, matrix and vector storage details, and output for status information.

4. **Main Execution Logic**
   - **Argument Reading**: Fetches command-line arguments to determine the number of equations and file names for matrix \( A \) and vector \( B \).
   - **Array Allocation**: Allocates memory for \( A \), \( B \), and pivot indices.
   - **File Reading**: Reads matrix \( A \) and vector \( B \) from files specified by the user.
   - **System Solving**: Calls `dgesv` to solve the linear system, where \( A \) is modified to store the factors from the LU decomposition, and \( B \) is overwritten with the solution \( X \).
   - **Output**: Writes the solution vector \( X \) back to a file.

5. **Error Handling**
   - Checks for successful allocation and reading operations, with appropriate error messages and program termination on failures.

#### Subroutine: `get_arguments`

- **Purpose**: Fetches and validates command-line arguments to ensure correct program setup.
- **Parameters**:
  - `nr_eqns`: Number of equations/system size.
  - `matrix_file_name`, `vector_file_name`: File paths for the matrix and vector data.

### Key Functionalities

- **Linear System Solver**: Uses `dgesv` for robust and efficient solving of linear systems, leveraging LAPACK's optimized routines.
- **Modular File Handling**: Employs modular functions from `linalg_mod` for input/output operations, promoting code reuse and organization.
- **Error Management**: Extensive error checking ensures the program gracefully handles common issues like invalid input or memory allocation failures.

### Usage

The program is intended to be executed with three command-line arguments specifying the number of equations, matrix file name, and vector file name. For example:
```bash
./solve_equations 3 matrix.dat vector.dat
```

This would solve a 3x3 system of equations using matrix and vector data from `matrix.dat` and `vector.dat`.

Next, let's proceed to read and explain the `linalg_mod.f90` file.

### Explaining `linalg_mod.f90`

The Fortran module `linalg_mod.f90` provides utility functions for reading, writing, and generating arrays. This module likely supports the functionality required by the other programs you uploaded (`generate_array.f90` and `solve_equations.f90`). Here's a breakdown of its contents and functionalities:

#### Module Structure and Functions

1. **Module `linalg_mod`**
   - Utilizes `iso_fortran_env` for standard environment definitions, such as `output_unit`, `error_unit`, and `DP` (double precision floating-point kind).

2. **Functionality Exposed**
   - `read_array`: Reads an array from a file whose dimensions and rank are defined by the inputs.
   - `write_array`: Writes an array to a file or standard output, handling both vectors and matrices.
   - `generate_random_array`: Fills an allocatable array with random numbers, based on the dimensions provided.

3. **Detailed Function Descriptions**
   - **`read_array`**:
     - Takes a file name, an array (allocatable), and its expected dimensions.
     - Checks if the rank of the array matches the number of dimensions provided.
     - Depending on the rank, it calls `read_vector` or `read_matrix` to perform the read operation.
     - Errors are managed by stopping the program if the ranks or dimensions do not match expectations.

   - **`write_array`**:
     - Accepts an array and an optional file name for output.
     - Uses rank-specific procedures (`write_vector` or `write_matrix`) to handle the output based on the array's rank.
     - If no file name is provided, the output is directed to the standard output (`output_unit`).

   - **`generate_random_array`**:
     - Designed to allocate and populate an array (either vector or matrix) with random numbers.
     - Checks the allocation status and initializes the array with random numbers using the intrinsic `random_number` procedure.
     - Manages errors related to mismatched ranks or failed allocations.

#### Subroutines for Vector and Matrix Handling

- **`read_vector` and `write_vector`**:
  - Handle reading and writing operations for vectors, respectively.

- **`read_matrix` and `write_matrix`**:
  - Handle reading and writing operations for matrices, respectively.

### Key Functionalities and Error Handling

- The module is equipped to handle errors related to input/output operations and array manipulations, which is critical for robust numerical software.
- Error messages are detailed, including specific issues like mismatched array ranks or failed memory allocations, which aids in debugging and user feedback.
- Flexibility to output to either a file or standard output enhances the module's usability across different computational scenarios.

### Usage

This module is utilized by other programs for managing arrays, particularly in numerical computations involving linear algebra. It abstracts away the details of file I/O and array initialization, allowing other parts of the software to focus on core computational logic.

Overall, `linalg_mod.f90` serves as a foundational component for array management in Fortran programs, providing essential services such as reading, writing, and generating arrays with a focus on error handling and user feedback.

## Program Code

The individual program files are listed below:

### In file linalg_mod.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/src/linalg_mod.f90
---
language: fortran
---
```

### In file generate_array.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/app/generate_array.f90
---
language: fortran
---
```

### In solve_equations.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/app/solve_equations.f90
---
language: fortran
---
```

The above programs are compiled and run using Fortran Package Manager (fpm):

## Build the Program using FPM (Fortran Package Manager)

In [1]:
import os
root_dir = ""
root_dir = os.getcwd()

Since the code makes use of the LAPACK library, the following FPM configuration file (fpm.toml) was used:

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/fpm.toml
---
language: toml
---
```

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_BLAS_LAPACK_Linear_Equations"

In [3]:
os.chdir(code_dir)

The files *solve_equations.f90* and *generate_array.f90* were placed into the "app" folder, while the file *linalg_mod.f90* was placed into the "src" folder.

In [4]:
build_status = os.system("fpm build 2>/dev/null")

## Run the Program using FPM (Fortran Package Manager)

### Solve a Test Linear System of Two Equations

As our first run, we wish to solve the following set of linear equations:

$$
\begin{align*}
2x+8y & = 20 \\
x+2y  & = 4
\end{align*}
$$

The variables in the equations are converted into components of the $\mathbf{x}$ vector as shown below:

$$
\begin{align*}
2x_1+8x_2 & = 20 \\
x_1+2x_2  & = 4
\end{align*}
$$

These equations are converted into matrix form as shown below:

$$
\begin{equation*}
\left[
\begin{array}{cc}
2 & 8 \\
1 & 2 \\
\end{array}
\right]
\left[
\begin{array}{c}
x_1 \\
x_2 \\
\end{array}
\right]
=
\left[
\begin{array}{c}
20 \\
4 \\
\end{array}
\right]
\end{equation*}
$$

Therefore we have the following:

$$
\mathbf{A} = 
\left[
\begin{array}{cc}
2 & 8 \\
1 & 2 
\end{array}
\right]
$$

$$
\mathbf{x} = 
\left[
\begin{array}{c}
x_1 \\
x_2  
\end{array}
\right]
$$


$$
\mathbf{b} = 
\left[
\begin{array}{c}
20 \\
4  
\end{array}
\right]
$$


The matrix $\mathbf{A}$ and the vector $\mathbf{b}$ are written into text files as shown below:

In [5]:
%%writefile A_test1.txt
2 8
1 2

Overwriting A_test1.txt


In [6]:
%%writefile b_test1.txt
20
4

Overwriting b_test1.txt


The *solve_equations* program can now be run with the number of equations command line argument set to 2, and the files *A_test1.txt* and *b_test1.txt*

In [7]:
exec_status = \
    os.system("fpm run solve_equations 2>/dev/null -- 2 A_test1.txt b_test1.txt")

         -2.000000000000000
          3.000000000000000


The results are printed in scientfic notation and in the order of $x_1$, $x_2$.

We now wish to use Python's Numpy library to test these results:

In [8]:
import numpy as np

A = np.genfromtxt("A_test1.txt")
b = np.genfromtxt("b_test1.txt")
x = np.linalg.solve(A, b)
print("x1 = {0:2.1f}".format(x[0]))
print("x2 = {0:2.1f}".format(x[1]))

x1 = -2.0
x2 = 3.0


We can see that the Fortran code and Numpy produce the same results.

### Solve a Test Linear System of Equations of Arbitrary Size

The Fortran code can be used to solve an arbitrarily large system of equations.  To test this functionality, we make use of the *generate_array.f90* program to generate arrays or matrices of arbitrary size filled with random numbers.

As a start, we will use the *generate_array.f90* to generate a matrix file A_test2.txt that contains a 10x10 matrix.

In [9]:
exec_status = os.system("fpm run generate_array 2>/dev/null -- 10 10 > A_test2.txt") 

The $\mathbf{A}$ matrix is shown below:

In [10]:
import pandas as pd
A = pd.read_table("A_test2.txt", 
    header=None, 
    sep='\s+')
A

,0,1,2,3,4,5,6,7,8,9
0,0.231881,0.224735,0.940890,0.839772,0.107850,0.253396,0.034549,0.090804,0.878374,0.063211
1,0.217384,0.580179,0.947205,0.753553,0.031057,0.752289,0.432489,0.350505,0.761701,0.679621
2,0.847354,0.704667,0.844039,0.799573,0.346590,0.526359,0.336583,0.691295,0.225961,0.534092
3,0.528730,0.084664,0.063896,0.516587,0.525722,0.782454,0.824814,0.697564,0.079712,0.086126
4,0.491231,0.900618,0.641910,0.851144,0.455506,0.625921,0.399938,0.658673,0.694518,0.003168
5,0.515378,0.552688,0.960681,0.932388,0.112611,0.036704,0.229438,0.408289,0.010820,0.197737
6,0.017810,0.072851,0.172264,0.627858,0.989993,0.859324,0.774557,0.607178,0.892918,0.862529
7,0.776959,0.097310,0.534678,0.857676,0.486515,0.439944,0.252270,0.564527,0.232126,0.608577
8,0.396415,0.941226,0.998837,0.276519,0.317670,0.087834,0.204389,0.400542,0.467099,0.711406
9,0.767424,0.847074,0.220408,0.546454,0.900786,0.642857,0.269771,0.818517,0.525927,0.995299


And now we generate the $\mathbf{b}$ vector:

In [11]:
exec_status = os.system("fpm run generate_array 2>/dev/null -- 10 > b_test2.txt") 

The $\mathbf{b}$ vector is shown below:

In [12]:
b = pd.read_table("b_test2.txt", 
    header=None, 
    sep='\s+')
b

,0
0,0.547368
1,0.188753
2,0.360579
3,0.431927
4,0.673998
5,0.885278
6,0.578111
7,0.162533
8,0.512780
9,0.419971


We now use the *solve_equation* Fortran code to solve this linear system of equations:

In [13]:
exec_status = \
    os.system("fpm run solve_equations 2>/dev/null -- 10 A_test2.txt b_test2.txt")

          1.119032007319357
          2.088738390676245
          0.299891011039355
          1.264642419542058
          3.718772323115945
          1.248192501462438
          1.837647345886853
         -7.556197620329637
         -1.810347214503931
         -0.478516612874785


And the results are compared to the output of Numpy:

In [14]:
A = np.genfromtxt("A_test2.txt")
b = np.genfromtxt("b_test2.txt")
x = np.linalg.solve(A, b)

for i in range(len(x)):
    print ("x{0:d} = {1:2.6f}".format(i+1, x[i]))

x1 = 1.119032
x2 = 2.088738
x3 = 0.299891
x4 = 1.264642
x5 = 3.718772
x6 = 1.248193
x7 = 1.837647
x8 = -7.556198
x9 = -1.810347
x10 = -0.478517


Again, we see that the results are the same.